In [1]:
from __future__ import division
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from difflib import SequenceMatcher

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [2]:
#import decisiontreeclassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
#import logisticregression classifier
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
#import knn classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#for validating your classification model
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# feature selection
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
df=pd.read_csv('fda_indi_react_predict.csv',low_memory=False)
df.head()

,indi_pt,drugname,age,gndr_cod,wt,de,lt,ho,ds,ca,ri,ot,pt
0,pain,ossicodone,79.67,0.0,61.0,0,0,0,0,0,0,1,abdominal pain
1,pain,ossicodone,79.67,0.0,61.0,0,0,0,0,0,0,1,blood creatinine increased
2,pain,ossicodone,79.67,0.0,61.0,0,0,0,0,0,0,1,diarrhoea
3,pain,ossicodone,79.67,0.0,61.0,0,0,0,0,0,0,1,pleural effusion
4,pain,ossicodone,79.67,0.0,61.0,0,0,0,0,0,0,1,weight decreased


In [4]:
len(df)

2186372

In [5]:
df2=df.sort_values(['indi_pt']).head(100000)

In [6]:
df_samp=df2.sample(frac=0.75, replace=False)

In [7]:
del(df)
import gc
gc.collect()
df=pd.DataFrame()

In [8]:
df_samp['pt'].value_counts()

no reaction                                    2617
nausea                                          810
pain                                            719
depression                                      646
anxiety                                         644
fatigue                                         629
drug ineffective                                612
headache                                        606
pyrexia                                         576
vomiting                                        574
diarrhoea                                       569
dyspnoea                                        556
dizziness                                       552
pneumonia                                       522
anaemia                                         492
inflammatory bowel disease                      445
insomnia                                        419
abdominal pain                                  412
off label use                                   412
febrile neut

In [9]:
df2_pneumonia=df_samp
df2_pneumonia['pneumonia']=0.0
df2_pneumonia.loc[df2_pneumonia['pt']=='pneumonia', 'pneumonia']=1
df2_pneumonia=df2_pneumonia.drop(['pt'], axis=1)
df2_pneumonia.head()

,indi_pt,drugname,age,gndr_cod,wt,de,lt,ho,ds,ca,ri,ot,pneumonia
837882,anticoagulant therapy,aspirin,65.00,0.0,52.00,0,0,0,0,0,0,0,0.0
940650,acne,accutane,15.00,1.0,65.80,0,0,0,1,0,0,0,0.0
346285,antiemetic supportive care,ondansetron 8mg,75.39,1.0,90.00,0,0,1,0,0,0,0,0.0
2152288,abdominal discomfort,protonix,78.00,0.0,57.10,0,0,1,0,0,0,0,1.0
649473,anxiety,propranolol,61.00,0.0,54.43,0,0,0,0,0,0,0,0.0


In [10]:
dummy_col = pd.get_dummies(df2_pneumonia[['indi_pt','drugname','gndr_cod']])
df2_pneumonia=df2_pneumonia.drop(['indi_pt','drugname','gndr_cod'] , axis=1)
df2_pneumonia = pd.concat([df2_pneumonia, dummy_col], axis=1)
df2_pneumonia.head()

,age,wt,de,lt,ho,ds,ca,ri,ot,pneumonia,...,drugname_zovirax /00587301/,drugname_zyclara,drugname_zyclara 3.75% medicis,drugname_zypadhera,drugname_zyprexa,drugname_zyprexa zidis,drugname_zyprexa zydis,drugname_zyrtec,drugname_zyrtec over the counter,drugname_zyvox
837882,65.00,52.00,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
940650,15.00,65.80,0,0,0,1,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
346285,75.39,90.00,0,0,1,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2152288,78.00,57.10,0,0,1,0,0,0,0,1.0,...,0,0,0,0,0,0,0,0,0,0
649473,61.00,54.43,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
y=df2_pneumonia['pneumonia']
X=df2_pneumonia.drop(['pneumonia','de','lt','ho','ds','ca','ri','ot'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
clf1 = RandomForestClassifier(n_estimators=50, class_weight='balanced',n_jobs=-1, oob_score=True)    #building 50 decision trees
clf1.fit(X_train, y_train)
print "pyrexia model"
print metrics.accuracy_score(y_test, clf1.predict(X_test))

print metrics.confusion_matrix(y_test, clf1.predict(X_test))

pyrexia model
0.9725333333333334
[[14577   319]
 [   93    11]]


In [12]:
lr= LogisticRegression(class_weight='balanced')
lr.fit(X_train, y_train)
print metrics.accuracy_score(y_test, lr.predict(X_test))
print metrics.confusion_matrix(y_test, lr.predict(X_test))
print metrics.classification_report(y_test, lr.predict(X_test))
print metrics.roc_auc_score(y_test, lr.predict(X_test))

0.8253333333333334
[[12335  2561]
 [   59    45]]
             precision    recall  f1-score   support

        0.0       1.00      0.83      0.90     14896
        1.0       0.02      0.43      0.03       104

avg / total       0.99      0.83      0.90     15000

0.6303834793026523


In [13]:
from imblearn.under_sampling import (ClusterCentroids, RandomUnderSampler,
                                     NearMiss,
                                     InstanceHardnessThreshold,
                                     CondensedNearestNeighbour,
                                     EditedNearestNeighbours,
                                     RepeatedEditedNearestNeighbours,
                                     AllKNN,
                                     NeighbourhoodCleaningRule,
                                     OneSidedSelection)

In [14]:
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=0)
ru=RandomUnderSampler(random_state=0)
X_resampled, y_resampled = ru.fit_sample(X_train, y_train)
#X_train_res, X_test_res, y_train_res, y_test_res = train_test_split(X_resampled, y_resampled, test_size=0.1, random_state=0, stratify=y)
clf2 = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1, random_state=0, oob_score=True)    #building 50 decision trees
clf2.fit(X_resampled, y_resampled)
print "pneumonia(pt) model (UNDERSAMPLING)"
print "oob score:", clf2.oob_score_
print metrics.accuracy_score(y_test, clf2.predict(X_test))
print metrics.confusion_matrix(y_test, clf2.predict(X_test))
print metrics.classification_report(y_test, clf2.predict(X_test))
print metrics.roc_auc_score(y_test, clf2.predict(X_test))

pneumonia(pt) model (UNDERSAMPLING)
oob score: 0.6351674641148325
0.6658666666666667
[[9915 4981]
 [  31   73]]
             precision    recall  f1-score   support

        0.0       1.00      0.67      0.80     14896
        1.0       0.01      0.70      0.03       104

avg / total       0.99      0.67      0.79     15000

0.6837690035528381


In [15]:
lr= LogisticRegression(class_weight='balanced')
lr.fit(X_resampled, y_resampled)
print metrics.accuracy_score(y_test, lr.predict(X_test))
print metrics.confusion_matrix(y_test, lr.predict(X_test))
print metrics.classification_report(y_test, lr.predict(X_test))
print metrics.roc_auc_score(y_test, lr.predict(X_test))

0.635
[[9453 5443]
 [  32   72]]
             precision    recall  f1-score   support

        0.0       1.00      0.63      0.78     14896
        1.0       0.01      0.69      0.03       104

avg / total       0.99      0.64      0.77     15000

0.6634537924481534
